In [1]:
import numpy as np
import pandas as pd
import sqlite3

In [2]:
import pandas as pd
import sqlite3

# Loading Datasets
orders = pd.read_csv('datasets/orders.csv')
users = pd.read_json('datasets/users.json')

conn = sqlite3.connect(":memory:")
cursor = conn.cursor()

In [3]:
# Execute SQL script
with open('datasets/restaurants.sql', 'r', encoding='utf-8') as f:
    sql_script = f.read()

cursor.executescript(sql_script)

# Load restaurants table
restaurants = pd.read_sql_query(
    "SELECT * FROM restaurants",
    conn
)

# Merge datasets
final_df = (
    orders
    .merge(users, on="user_id", how="left")
    .merge(restaurants, on="restaurant_id", how="left")
)

# Save output
final_df.to_csv("datasets/final_food_delivery_dataset.csv", index=False)

In [4]:
df = pd.read_csv('datasets/final_food_delivery_dataset.csv')

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   order_id           10000 non-null  int64  
 1   user_id            10000 non-null  int64  
 2   restaurant_id      10000 non-null  int64  
 3   order_date         10000 non-null  object 
 4   total_amount       10000 non-null  float64
 5   restaurant_name_x  10000 non-null  object 
 6   name               10000 non-null  object 
 7   city               10000 non-null  object 
 8   membership         10000 non-null  object 
 9   restaurant_name_y  10000 non-null  object 
 10  cuisine            10000 non-null  object 
 11  rating             10000 non-null  float64
dtypes: float64(2), int64(3), object(7)
memory usage: 937.6+ KB


In [6]:
df.head()

,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name_x,name,city,membership,restaurant_name_y,cuisine,rating
0,1,2508,450,18-02-2023,842.97,New Foods Chinese,User_2508,Hyderabad,Regular,Restaurant_450,Mexican,3.2
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine,User_2693,Pune,Regular,Restaurant_309,Indian,4.5
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi,User_2084,Chennai,Gold,Restaurant_107,Mexican,4.0
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg,User_319,Bangalore,Gold,Restaurant_224,Chinese,4.8
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian,User_1064,Pune,Regular,Restaurant_293,Italian,3.0


In [7]:
df.isnull().sum()

,0
order_id,0
user_id,0
restaurant_id,0
order_date,0
total_amount,0
restaurant_name_x,0
name,0
city,0
membership,0
restaurant_name_y,0


In [8]:
df.duplicated().sum()

np.int64(0)

In [9]:
# Which city has the highest total revenue (total_amount) from Gold members?
df[df['membership']=='Gold'].groupby('city')['total_amount'].sum().sort_values(ascending=False)

,total_amount
city,
Chennai,1080909.79
Pune,1003012.32
Bangalore,994702.59
Hyderabad,896740.19


In [10]:
# Which cuisine has the highest average order value across all orders?
df.groupby("cuisine")["total_amount"].mean().sort_values(ascending=False)

,total_amount
cuisine,
Mexican,808.021344
Italian,799.448578
Indian,798.466011
Chinese,798.389020


In [11]:
# How many distinct users placed orders worth more than ₹1000 in total (sum of all their orders)?
(df.groupby("user_id")["total_amount"].sum() > 1000).sum()

np.int64(2544)

In [12]:
# Which restaurant rating range generated the highest total revenue?
bins = [3.0, 3.5, 4.0, 4.5, 5.0]
labels = ["3.0–3.5", "3.6–4.0", "4.1–4.5", "4.6–5.0"]

df["rating_range"] = pd.cut(df["rating"], bins=bins, labels=labels, include_lowest=True)

df.groupby("rating_range")["total_amount"].sum().sort_values(ascending=False)


/tmp/ipython-input-2219512577.py:7: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df.groupby("rating_range")["total_amount"].sum().sort_values(ascending=False)


,total_amount
rating_range,
4.6–5.0,2197030.75
3.0–3.5,2136772.70
4.1–4.5,1960326.26
3.6–4.0,1717494.41


In [13]:
# Which cuisine has fewest restaurants but high revenue?
(df.groupby("cuisine")["total_amount"].sum() /
 df.groupby("cuisine")["restaurant_id"].nunique()) .sort_values(ascending=False)

,0
cuisine,
Mexican,16292.992891
Chinese,16087.538750
Italian,16065.109524
Indian,15646.131587


In [14]:
# What percentage of total orders were placed by Gold members? (Rounded to nearest integer)
round((df[df["membership"] == "Gold"].shape[0] / df.shape[0]) * 100)

50

In [15]:
# Which restaurant has the highest average order value but less than 20 total orders?
restaurant_stats = df.groupby(["restaurant_name_y",'restaurant_name_x']).agg(
    avg_order=("total_amount", "mean"),
    orders=("order_id", "count"))

restaurant_stats[restaurant_stats["orders"] < 20].sort_values("avg_order", ascending=False).head()

,,avg_order,orders
restaurant_name_y,restaurant_name_x,,
Restaurant_294,Hotel Dhaba Multicuisine,1040.222308,13
Restaurant_262,Hotel Cafe Punjabi,1029.473333,18
Restaurant_77,Sri Mess Punjabi,1029.180833,12
Restaurant_193,Swagath Eatery Multicuisine,1026.306667,15
Restaurant_7,Ruchi Biryani Punjabi,1002.140625,16


In [16]:
# Which membership + cuisine combination contributes the highest revenue?
df.groupby(["membership", "cuisine"])["total_amount"].sum().sort_values(ascending=False)

membership  cuisine
Regular     Mexican    1072943.30
            Italian    1018424.75
Gold        Mexican    1012559.79
            Italian    1005779.05
Regular     Indian      992100.27
Gold        Indian      979312.31
            Chinese     977713.74
Regular     Chinese     952790.91
Name: total_amount, dtype: float64

In [17]:
# During which quarter is total revenue highest?
df['order_date'] = pd.to_datetime(df['order_date'], format='%d-%m-%Y')
df["quarter"] = df["order_date"].dt.to_period("Q")

df.groupby("quarter")["total_amount"].sum() \
  .sort_values(ascending=False)

,total_amount
quarter,
2023Q3,2037385.10
2023Q4,2018263.66
2023Q1,1993425.14
2023Q2,1945348.72
2024Q1,17201.50


In [18]:
# How many total orders were placed by users with Gold membership?
df[df['membership']=='Gold'].shape[0]

4987

In [19]:
# What is the total revenue (rounded to nearest integer) generated from orders placed in Hyderabad city?
round((df[df["city"] == "Hyderabad"]['total_amount']).sum())

1889367

In [20]:
# How many distinct users placed at least one order?
df['user_id'].nunique()

2883

In [21]:
# What is the average order value (rounded to 2 decimals) for Gold members?
round(df[df['membership']=='Gold']['total_amount'].mean(),2)

np.float64(797.15)

In [22]:
# How many orders were placed for restaurants with rating ≥ 4.5?
df[df['rating']>=4.5].shape[0]

3374

In [23]:
# How many orders were placed in the top revenue city among Gold members only?
df[df['membership']=='Gold']['city'].value_counts().head(1)

,count
city,
Chennai,1337
